This notebook contains the commands to export selected visualizations as HTML.

Just run this notebook to save the visualizations as interactive HTML maps. Use the [visualize](./visualize.ipynb) notebook if you want to learn how to create dataframes and visualizations from PostGIS data.

In [1]:
import sys
from ipygis import get_connection_url, QueryResult, get_map, generate_map
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from kepler_h3_config import config  # we may use our own custom visualization config

This variable changes the city that we wish to plot. Data for different cities are in different database schemas:

In [2]:
city = 'helsinki'

In [3]:
sql_url = get_connection_url(dbname='geoviz')
engine = create_engine(sql_url)
schema_engine = engine.execution_options(
    schema_translate_map={'schema': city}
)
session = sessionmaker(bind=schema_engine)()

sys.path.insert(0, '..')
from models import OSMPoint, FlickrPoint, GTFSStop, OSMAccessNode, OoklaPoint, KonturPoint
from osm_tags import tag_filter

The database queries that make up our analysis are

In [4]:
places_query = session.query(OSMPoint).filter(tag_filter)
photos_query = session.query(FlickrPoint)
stops_query = session.query(GTFSStop)
access_points_query = session.query(OSMAccessNode)
ookla_query = session.query(OoklaPoint)
kontur_query = session.query(KonturPoint)

You may visualize all points in any database query by simply

In [5]:
get_map(ookla_query)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'data_1', 'type': '…

Datasets
--------

The next phase will take a while. Here we actually create all the dataframes in memory (using H3 hex resolution 8), from the whole database.

The **places** dataset simply sums the *number of amenity POIs* in each H3 hex. The larger areas you have imported, the longer this query will take:

In [6]:
places = QueryResult.create(places_query, resolution=8, name='places')

The **photographers** dataset calculates the *number of people taking photos* in each H3 hex. Grouping the points by hex and photographer takes a while:

In [9]:
photographers = QueryResult.create(photos_query, resolution=8, name='photographers', group_by='properties.owner')

The **trips** dataset calculates the *number of trips leaving from all transit stops* in the H3 hex in a single day. This shouldn't take too long:

In [10]:
trips = QueryResult.create(stops_query, resolution=8, name='trips', plot='sum', column='properties.ntrips')

The **access** dataset calculates the *average walking time to 5th closest amenity* in each H3 hex. The amount of walking nodes is quite big so this is a slow query:

In [11]:
access = QueryResult.create(access_points_query, resolution=8, name='access', plot='mean', column='accessibilities.5')

The **devices** dataset calculates the *number of broadband devices* making speedtests in each H3 hex. (Alternatively, we could consider the average broadband speeds in the hex.) This should be fast:

In [12]:
speeds = QueryResult.create(ookla_query, resolution=8, name='speeds', plot='mean', column='properties.avg_d_kbps')
devices = QueryResult.create(ookla_query, resolution=8, name='devices', plot='sum', column='properties.devices')

The **population** dataset contains the *approximate population* in the H3 hex. This should be fast:

In [13]:
population = QueryResult.create(kontur_query, resolution=8, name='population', plot='sum', column='properties.population')

Visualization
-------------

Once the QueryResults exist, calculating the sum map from dataframes should be relatively fast. Layer weights in the sum may be customized here if you want to emphasize or leave out some datasets.

(Note that access layer has *negative* weight, because smaller walking times indicate better pedestrian access to amenities.)

In [14]:
results = [photographers, trips, access, devices, population]
columns = ['size', 'ntrips', '5', 'devices', 'population']
weights = [1, 1, -1, 1, 1]

In [15]:
result_map = generate_map(results, 500, config=config, column=columns, weights=weights, clusters=0.005)
result_map.save_to_html(file_name=f"../server/maps/{city}.html")
result_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter

Global spatial autocorrelation:
Moran's I:   0.848
p-value:     0.001

Distribution of clusters/outliers (quadrants):
LL    612
HH    418
HL     48
LH     45
Name: quadrant, dtype: int64

Significant clusters (using significance threshold 0.005):
False    781
True     342
Name: significant, dtype: int64
Map saved to ../server/maps/copenhagen.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['clusters'], 'id': '0i7a3fmz…

In [8]:
devices

In [9]:
devices.gdf

,devices,geometry,hex8,normalized
hex8,,,,
881f0061c7fffff,73,POINT (12.1431975895452791 54.0865820495134031),881f0061c7fffff,0.088916
881f0061c1fffff,35,POINT (12.1399088603757868 54.0935499759708023),881f0061c1fffff,0.042631
881f00635dfffff,12,POINT (12.0479750857779262 54.1565385985282504),881f00635dfffff,0.014616
881f006155fffff,16,POINT (12.0743635955596194 54.1008591448480018),881f006155fffff,0.019488
881f0061c5fffff,54,POINT (12.1300897800501790 54.0880476627352280),881f0061c5fffff,0.065773
...,...,...,...,...
8808996a97fffff,3,POINT (24.7283136407836253 60.2597615958787856),8808996a97fffff,0.003654
881126d76bfffff,3,POINT (24.9998813680096639 60.2873336420900472),881126d76bfffff,0.003654
881126d56bfffff,4,POINT (25.0925628931148381 60.2750859106223373),881126d56bfffff,0.004872


In [26]:
len(photographers.gdf)

9129